<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/Nerfstudiodemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# ネルフスタジオNeRFのためのコラボレーション・フレンドリーなスタジオ


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

ネルフスタジオNeRFのためのコラボレーション・フレンドリーなスタジオ
GitHub stars

本コラボでは、NerfstudioのNeRFを既成のデータセットや自分のビデオ・画像から学習・閲覧する方法を紹介します。



Google Colabのフォーマットについては、NeX社にクレジットをお願いしています

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

# **よくある質問**
* **カスタムデータのダウンロードが止まっている（出力されない）**
*- Colabのバグです。
データは処理されていますが、完了までに時間がかかる場合があります。data/nerfstudio/custom_data/transforms.jsonが存在すれば処理が完了したことになります。
*- 早期にセルを終了させると、学習ができなくなります。
* **カスタムデータの処理に時間がかかっている。**
*- データの処理にかかる時間は、画像の枚数とその解像度によって異なります。
*- 処理に時間がかかりすぎる場合は、カスタムデータの解像度を下げてみてください。
- **エラーです。データ処理が完了しませんでした。**
-* これは、データ処理スクリプトが完全に完了しなかったことを意味します。
-*これは、画像の枚数が少ないか、画像の品質が低いことが原因である可能性があります。
-* 処理に成功する可能性を高めるために、モーションブラーがほとんどなく、シーンの視覚的な重なりが多い画像をお勧めします。
- **トレーニングの進捗が表示されない。**
-* 出力されないのは、Colabのバグです。ビューアからトレーニングの進捗を確認することができます。
- **ビューアの品質が悪い / 解像度が低い**
*- これは、ビューアをレンダリングするために、トレーニングに多くのGPUが使用されていることが原因である可能性があります。
*- トレーニングを一時停止するか、トレーニングの使用率を下げてみてください。
- **警告: 'root'ユーザーとしてpipを実行しています...::**
-* この警告とその他のpipの警告やエラーは、安全に無視してかまいません。
-**その他の問題？**
-* GitHub リポジトリにイシューを作成してください。

In [ ]:
#@markdown Condaのインストール(ランタイムの再起動が必要)</h1>

!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@markdown # Nerfstudioのインストール (約10分)

%cd /content/
!pip install --upgrade pip
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# TinyCudaのインストール
%cd /content/
!gdown "https://drive.google.com/u/1/uc?id=1q8fuc-Mqiev5GTBTRA5UPgCaQDzuqKqj" 
!pip install tinycudann-1.6-cp37-cp37m-linux_x86_64.whl

# COLMAPのインストール
%cd /content/
!conda install -c conda-forge colmap

# nerfstudioのインストール
%cd /content/
# !pip install nerfstudio
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

In [ ]:
#@markdown データのダウンロードと加工
#@markdown 前もって調整・定義された設置値のシーンを選ぶか、自分の画像や動画をアップロードする
import os
import glob
from google.colab import files
from IPython.core.display import display, HTML

scene = '\uD83D\uDDBC poster' #@param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = ' '.join(scene.split(' ')[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("エラー、Polycam データの処理中に単一の .zip ファイルをアップロードしてください")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == 'upload your own Record3D data':
    display(HTML('<h3>Zip your Record3D folder, and upload.</h3>'))
    display(HTML('<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'))
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob('/content/data/nerfstudio/custom_data/*')[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ['upload your images', 'upload your own video']:
    display(HTML('<h3>Select your custom data</h3>'))
    display(HTML('<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>'))
    display(HTML('<p/>Note: This may take time, especially on hires inputs, so we recommend to download dataset after creation.<p>'))
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == 'upload your images':
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == 'upload your images':
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("データ処理成功!")

In [ ]:
#@markdown 観測者の設定と起動

%cd /content

# localtunnelのインストール
# localtunnel https://github.com/localtunnel/localtunnelを使用していますが、ngrokも使用可能です。
!npm install -g localtunnel

# デフォルトであるトンネルポート7007
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) #前のコマンドは url.txt に書き込む時間が必要です。


with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
# from nerfstudio.utils.io import load_from_json
# from pathlib import Path
# json_filename = "nerfstudio/nerfstudio/viewer/app/package.json"
# version = load_from_json(Path(json_filename))["version"]
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
print("トレーニングを開始したら、ページの更新をクリックする必要があるかもしれません！")
from IPython import display
display.IFrame(src=url, height=800, width="100%")

In [ ]:
#@markdown トレーニング開始

%cd /content
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    !ns-train nerfacto --viewer.websocket-port 7007 nerfstudio-data --data data/nerfstudio/$scene --downscale-factor 4
else:
    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML('<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>'))

In [ ]:
#@title # ビデオをレンダリング { vertical-output:true } { vertical-output: true }
#@markdown 視点内からカメラパスをエクスポート(特定のアプリケーションで読み込むことができるようなファイルの形式に出力する作業)し、このセルを実行します
#@markdown レンダリングされた動画は、renders/output.mp4 にあるはずです。


base_dir = "/content/outputs/data-nerfstudio-" + scene + "/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import display, HTML
display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render --load-config $config_filename --traj filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4